In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
import gc

## Data Prep

In [2]:
# train_X = pd.read_csv('music_data/train_X.csv')
train_X = pd.read_csv('music_data/train_X_date_converted.csv')
train_y = pd.read_csv('music_data/train_Y.csv')

# val_X = pd.read_csv('music_data/valid_X.csv')
val_X = pd.read_csv('music_data/valid_X_date_converted.csv')
val_y = pd.read_csv('music_data/valid_Y.csv')

# test_X = pd.read_csv('music_data/test_X.csv')
test_X = pd.read_csv('music_data/test_X_date_converted.csv')
test_y = pd.read_csv('music_data/test_Y.csv')

# train_tiny_X = pd.read_csv('music_data/train_tiny_X.csv')
train_tiny_X = pd.read_csv('music_data/train_tiny_X_date_converted.csv')
train_tiny_y = pd.read_csv('music_data/train_tiny_Y.csv')

# val_tiny_X = val_X[:1000].copy()
val_tiny_X = pd.read_csv('music_data/val_tiny_X_date_converted.csv')
val_tiny_y = val_y[:1000].copy()

In [4]:
from sklearn.feature_selection import SelectKBest
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numCols),
                                               ('cat', categorical_transformer, catCols)])
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(penalty='l1'))])

clf.fit(train_tiny_X, train_tiny_y['target'])
print("model score: %.3f" % clf.score(val_tiny_X, val_tiny_y['target']))
print(classification_report(train_tiny_y['target'], clf.predict(train_tiny_X)))
print(roc_auc_score(train_tiny_y['target'], clf.predict_proba(train_tiny_X)[:,1]))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


model score: 0.660
              precision    recall  f1-score   support

         0.0       0.72      0.69      0.71      4985
         1.0       0.71      0.73      0.72      5015

   micro avg       0.71      0.71      0.71     10000
   macro avg       0.71      0.71      0.71     10000
weighted avg       0.71      0.71      0.71     10000

0.7882886145975314


/opt/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


## Random Forest

In [3]:
catCols = ['msno', 'song_id', 'source_screen_name', 'source_system_tab', 'source_type', 'genre_ids', 'artist_name', 
           'composer', 'lyricist', 'language', 'city', 'gender', 'registered_via']
numCols = ['bd', 'song_length']

In [5]:
def get_column_names_from_ColumnTransformer(column_transformer):    
    col_name = []
    for transformer_in_columns in column_transformer.transformers_[:]:#the last transformer is ColumnTransformer's 'remainder'
        raw_col_name = transformer_in_columns[2]
        if isinstance(transformer_in_columns[1],Pipeline): 
            transformer = transformer_in_columns[1].steps[-1][1]
        else:
            transformer = transformer_in_columns[1]
        try:
            names = transformer.get_feature_names()
        except AttributeError: # if no 'get_feature_names' function, use raw column name
            names = raw_col_name
        if isinstance(names,np.ndarray): # eg.
            col_name += names.tolist()
        elif isinstance(names,list):
            col_name += names    
        elif isinstance(names,str):
            col_name.append(names)
    return col_name

In [6]:
coef = clf.named_steps['classifier'].coef_
feature_names = get_column_names_from_ColumnTransformer(clf.named_steps['preprocessor'])
for i in range(len(feature_names)):
    if feature_names[i][0] == 'x':
        flag = i
        break
cat_names = feature_names[flag:]
select = []
for i in range(coef.shape[1]):
    if coef[0,i] != 0:
        select.append(i)
select_features = [feature_names[i] for i in select]
len(select_features)

1401

In [7]:
len(feature_names)

21102

In [8]:
for i in range(len(select_features)):
    if select_features[i][0] == 'x':
        flag = i
        break

In [9]:
num_select = select_features[:flag]
cat_select = select_features[flag:]
inds = []
for i in range(len(cat_select)):
    inds.append(cat_names.index(cat_select[i]))

In [16]:
data_scaler = StandardScaler()
train_num = data_scaler.fit_transform(train_tiny_X[numCols])
val_num = data_scaler.transform(val_tiny_X[numCols])
train_num_df = pd.DataFrame(train_num,columns = numCols)[num_select]
train_num_df = train_num_df.reset_index(drop=True)
val_num_df = pd.DataFrame(val_num,columns = numCols)[num_select]
val_num_df = val_num_df.reset_index(drop=True)

train_cat = train_tiny_X[catCols]
one_hot = OneHotEncoder(handle_unknown='ignore')
one_hot.fit(train_cat)
feature_names = [str(i) for i in list(one_hot.get_feature_names())]
for i in range(4):
    if i == 0:
        train_piece = one_hot.transform(train_cat[i*1000:(i+1)*1000]).toarray()[:,inds]
        val_piece = one_hot.transform(val_tiny_X[catCols]).toarray()[:,inds]
    elif i == 3:
        train_piece = np.vstack((train_piece,one_hot.transform(train_cat[3000:]).toarray()[:,inds]))      
    else:
        train_piece = np.vstack((train_piece,one_hot.transform(train_cat[i*1000:(i+1)*1000]).toarray()[:,inds]))

train_cat_df = pd.DataFrame(train_piece,columns = cat_select)
val_cat_df = pd.DataFrame(val_piece,columns = cat_select)
newcatCols = list(train_cat_df.columns)

try_train_X = pd.concat([train_num_df,train_cat_df],axis = 1)
try_val_X = pd.concat([val_num_df,val_cat_df],axis = 1)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
from sklearn.multioutput import MultiOutputClassifier
train_tiny_y.head()

,Unnamed: 0,target
0,2942719,0.0
1,4875524,0.0
2,6589819,0.0
3,1172060,1.0
4,2069395,0.0


In [47]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 500, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf_random = RandomizedSearchCV(estimator = RandomForestClassifier(random_state=0), param_distributions = random_grid, n_iter = 20, cv = 3, verbose=1, random_state=0)
# rf_random.fit(try_train_X, train_tiny_y)

In [50]:
rf_random.fit(try_train_X, train_tiny_y['target'])

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  9.8min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [51]:
print(rf_random.best_params_)

{'n_estimators': 233, 'min_samples_split': 10, 'max_features': 'auto', 'max_depth': 70, 'min_samples_leaf': 1, 'bootstrap': False}


In [74]:
clf = RandomForestClassifier(n_estimators=233,random_state=0,min_samples_split=10,max_features='auto', max_depth=70,min_samples_leaf=1, bootstrap=False)
clf.fit(try_train_X, train_tiny_y['target'])
print(classification_report(val_tiny_y['target'], clf.predict(try_val_X)>0.5))
print(roc_auc_score(val_tiny_y['target'], clf.predict(try_val_X)))

              precision    recall  f1-score   support

         0.0       0.61      0.64      0.62       477
         1.0       0.66      0.63      0.64       523

    accuracy                           0.64      1000
   macro avg       0.63      0.64      0.63      1000
weighted avg       0.64      0.64      0.64      1000

0.6351018755687033


## Random Forest without Categorical Variables

In [76]:
train_mca_tiny_X = pd.read_csv('music_data/mca_train_tiny_X.csv')
train_mca_tiny_y = pd.read_csv('music_data/mca_train_tiny_y.csv')

val_mca_tiny_X = pd.read_csv('music_data/mca_val_tiny_X.csv')
val_mca_tiny_y = pd.read_csv('music_data/mca_val_tiny_y.csv')

In [18]:
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 500, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf_random = RandomizedSearchCV(estimator = RandomForestClassifier(random_state=0), param_distributions = random_grid, n_iter = 20, cv = 3, verbose=1, random_state=0)
rf_random.fit(train_mca_tiny_X, train_mca_tiny_y['target'])

NameError: name 'train_mca_tiny_X' is not defined

In [78]:
print(rf_random.best_params_)

{'n_estimators': 286, 'min_samples_split': 5, 'max_features': 'sqrt', 'max_depth': 40, 'min_samples_leaf': 4, 'bootstrap': True}


In [88]:
clf_mca = RandomForestClassifier(verbose=2,n_estimators=286,random_state=0,min_samples_split=5,max_features='sqrt', max_depth=40,min_samples_leaf=4, bootstrap=True)
clf_mca.fit(train_mca_tiny_X, train_mca_tiny_y['target'])
print(classification_report(val_mca_tiny_y['target'], clf_mca.predict(val_mca_tiny_X)>0.5))
print(roc_auc_score(val_mca_tiny_y['target'], clf_mca.predict(val_mca_tiny_X)))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


building tree 1 of 286
building tree 2 of 286
building tree 3 of 286
building tree 4 of 286
building tree 5 of 286
building tree 6 of 286
building tree 7 of 286
building tree 8 of 286
building tree 9 of 286
building tree 10 of 286
building tree 11 of 286
building tree 12 of 286
building tree 13 of 286
building tree 14 of 286
building tree 15 of 286
building tree 16 of 286
building tree 17 of 286
building tree 18 of 286
building tree 19 of 286
building tree 20 of 286
building tree 21 of 286
building tree 22 of 286
building tree 23 of 286
building tree 24 of 286
building tree 25 of 286
building tree 26 of 286
building tree 27 of 286
building tree 28 of 286
building tree 29 of 286
building tree 30 of 286
building tree 31 of 286
building tree 32 of 286
building tree 33 of 286
building tree 34 of 286
building tree 35 of 286
building tree 36 of 286
building tree 37 of 286
building tree 38 of 286
building tree 39 of 286
building tree 40 of 286
building tree 41 of 286
building tree 42 of 286
b

[Parallel(n_jobs=1)]: Done 286 out of 286 | elapsed:   10.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 286 out of 286 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 286 out of 286 | elapsed:    0.1s finished


## Scale Up on the Large Dataset

In [12]:
new_train_X = pd.read_csv('new_data/new_train_X.csv')
new_train_y = pd.read_csv('new_data/new_train_y.csv')

new_valid_X = pd.read_csv('new_data/new_valid_X.csv')
new_valid_y = pd.read_csv('new_data/new_valid_y.csv')

new_test_X = pd.read_csv('new_data/new_test_X.csv')
new_test_y = pd.read_csv('new_data/new_test_y.csv')

In [13]:
new_train_y.head()

,target
0,0.0
1,1.0
2,0.0
3,1.0
4,1.0


In [14]:
new_train_y.rename(columns={'0.0':'target'})
new_valid_y.rename(columns={'0.0':'target'})
new_valid_y.rename(columns={'0.0':'target'})

,target
0,1.0
1,1.0
2,0.0
3,1.0
4,1.0
5,1.0
6,0.0
7,1.0
8,0.0
9,1.0


In [17]:
data_scaler = StandardScaler()
train_num = data_scaler.fit_transform(new_train_X[numCols])
val_num = data_scaler.transform(new_valid_X[numCols])
train_num_df = pd.DataFrame(train_num,columns = numCols)[num_select]
train_num_df = train_num_df.reset_index(drop=True)
val_num_df = pd.DataFrame(val_num,columns = numCols)[num_select]
val_num_df = val_num_df.reset_index(drop=True)

train_cat = new_train_X[catCols]
one_hot = OneHotEncoder(handle_unknown='ignore')
one_hot.fit(train_cat)
feature_names = [str(i) for i in list(one_hot.get_feature_names())]
for i in range(4):
    if i == 0:
        train_piece = one_hot.transform(train_cat[i*1000:(i+1)*1000]).toarray()[:,inds]
        val_piece = one_hot.transform(new_valid_X[catCols]).toarray()[:,inds]
    elif i == 3:
        train_piece = np.vstack((train_piece,one_hot.transform(train_cat[3000:]).toarray()[:,inds]))      
    else:
        train_piece = np.vstack((train_piece,one_hot.transform(train_cat[i*1000:(i+1)*1000]).toarray()[:,inds]))

train_cat_df = pd.DataFrame(train_piece,columns = cat_select)
val_cat_df = pd.DataFrame(val_piece,columns = cat_select)
newcatCols = list(train_cat_df.columns)

new_train_X = pd.concat([train_num_df,train_cat_df],axis = 1)
new_valid_X = pd.concat([val_num_df,val_cat_df],axis = 1)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


MemoryError: 

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 500, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf_random = RandomizedSearchCV(estimator = RandomForestClassifier(random_state=0), param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=0)
# rf_random.fit(try_train_X, train_tiny_y)

In [12]:
gc.collect()

63